In [1]:
import torch

In [2]:
from colorama import Back, Style

TEXT2ID = {
    "O": 0,
    "B-AC": 1,
    "B-LF": 2,
    "I-LF": 3,
}

def vizu(words, output, type=None):
    sentence = " ".join(words)
    out_words = []
    out_label = []
    index = 1
    for i in range(len(output)):
        start = output[i]['start']
        end = output[i]['end']
        word = output[i]['word']
        if word[0] != '#':
            out_words.append(' ')
            out_label.append(0)
            index += 1
        out_words.append(sentence[start:end])
        if type==1:
            out_label.append(TEXT2ID[output[i]['entity']])
        else:
            out_label.append(int(output[i]['entity'][-1]))
    col = {0: Back.BLACK, 1: Back.RED, 2: Back.GREEN, 3: Back.BLUE, 4: Back.MAGENTA}
    for i in range(len(out_words)):
        print(col[out_label[i]], end='')
        print(out_words[i], end='')
        print(Style.RESET_ALL, end='')
    print()

In [3]:
import transformers
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
from datasets import load_dataset
datasets = load_dataset("surrey-nlp/PLOD-CW")
TEXT2ID = {
    "B-O": 0,
    "B-AC": 1,
    "B-LF": 2,
    "I-LF": 3,
}
datasets = datasets.map(lambda x: {"ner_tags": [TEXT2ID[tag] for tag in x["ner_tags"]]})
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if True else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)
from transformers import AutoModelForTokenClassification

In [4]:
path = 'checkpoints/checkpoint-1200'
path = 'surrey-nlp/roberta-base-finetuned-abbr'
model = AutoModelForTokenClassification.from_pretrained(path, num_labels=5)

In [5]:
pipeline = transformers.pipeline("ner", model=model, tokenizer=tokenizer, ignore_labels=[])

def choose(i=None):
    if i is None:
        i = torch.randint(0, len(datasets["test"]["tokens"]), (1,)).item()
    output = pipeline(" ".join(datasets["test"]["tokens"][i]))
    words = datasets["test"]["tokens"][i]

    return words, output

def choose_multiple(nb=5):
    indices = torch.randint(0, len(datasets["test"]["tokens"]), (nb,))
    words = []
    outputs = []
    for i in indices:
        w, o = choose(i)
        words.append(w)
        outputs.append(o)
    return words, outputs

In [6]:
words, outputs = choose_multiple()
for i in range(len(words)):
    vizu(words[i], outputs[i], type=1)

KeyError: 'O'